# Grupo 3: Trabajo Final Integrador

Churichi Alan, Gonzalez Exequiel, Mare Rodrigo

In [1]:
import roboticstoolbox as rtb
import numpy as np
import sim
import sys
import time
import pickle
from handlers import *
from kinematics import *
from utils import *
from trajectories import *
from constants import *
from movement import *

### Movimiento del robot utilizando el MCI calculado

In [2]:
clientID = connect(19999)

Conectado a  19999


In [3]:
clientID = connect(19999)

t0 = 0;
if (clientID != -1): # Mientras haya conexión con Coppelia
    joints_R1 = [0 for i in range(JOINTS_NUM)] # ID de las articulaciones
    joints_R2 = [0 for i in range(JOINTS_NUM)] # ID de las articulaciones
    home_R1 = {}
    home_R2 = {}
    checkpoints_move_box = [{} for i in range(CHECKPOINTS_R1_NUM)] # Checkpoints del robot 1 para mover la caja
    checkpoints_pioneer = [{} for i in range(CHECKPOINTS_PIONEER_NUM)] # Checkpoints para el robot 1 para poder ubicar las cajas en el pioneer
    checkpoints_boxes = [{} for i in range(CHECKPOINTS_BOXES_NUM)] # Checkpoints de las cajas en la cinta
    checkpoints_boxes_shelf = [{} for i in range(CHECKPOINTS_PIONEER_NUM)] # Checkpoints de las cajas en pioneer cuando está en la estantería
    checkpoints_store_box = [{} for i in range(CHECKPOINTS_R2_NUM)] # Checkpoints para mover las cajas en la estantería
    checkpoints_shelf = [{} for i in range(CHECKPOINTS_SHELF)] # Checkpoints estantes
    
    # Se obtienen los IDs
    joint_handlers(joints_R1, 'R1', JOINTS_NUM, clientID, sim.simx_opmode_blocking)
    joint_handlers(joints_R2, 'R2', JOINTS_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_move_box, 'CheckPoint_R1', CHECKPOINTS_R1_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_pioneer, 'CheckPoint_box_pioneer', CHECKPOINTS_PIONEER_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_boxes, 'CheckPoint_box', CHECKPOINTS_BOXES_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_boxes_shelf, 'CheckPoint_box_pioneer_shelf', CHECKPOINTS_PIONEER_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_store_box, 'CheckPoint_R2', CHECKPOINTS_R2_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_shelf, 'CheckPoint_shelf', CHECKPOINTS_SHELF, clientID, sim.simx_opmode_blocking)
    _, tip_R1 = sim.simxGetObjectHandle(clientID, (ROBOT_NAME + '_tip'), sim.simx_opmode_blocking)
    _, tip_R2 = sim.simxGetObjectHandle(clientID, (ROBOT_NAME + '_tip_2'), sim.simx_opmode_blocking)
    _, sensor_1 = sim.simxGetObjectHandle(clientID, 'Proximity_sensor_box_low', sim.simx_opmode_blocking)
    _, sensor_2 = sim.simxGetObjectHandle(clientID, 'Proximity_sensor_box_high', sim.simx_opmode_blocking)

    # Se setean las posiciones y orientaciones como streaming
    sim.simxGetObjectPosition(clientID, tip_R1, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectOrientation(clientID, tip_R1, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectPosition(clientID, tip_R2, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectOrientation(clientID, tip_R2, -1, sim.simx_opmode_streaming)
    checkpoints_set_streaming([checkpoints_move_box, checkpoints_pioneer, checkpoints_boxes, checkpoints_boxes_shelf, checkpoints_store_box, checkpoints_shelf], clientID)
    sim.simxReadProximitySensor(clientID, sensor_1, sim.simx_opmode_streaming)
    sim.simxReadProximitySensor(clientID, sensor_2, sim.simx_opmode_streaming)

    # Se inicia la simulación
    sim.simxStartSimulation(clientID, sim.simx_opmode_oneshot)
    # sim.simxPauseSimulation(clientID, sim.simx_opmode_oneshot)
    
    time.sleep(1) # Tiempo de espera para asegurarse que los valores leidos de home sean correctos

    # Posición, orientación y ángulos de Home
    _, home_R1['position'] = sim.simxGetObjectPosition(clientID, tip_R1, -1, sim.simx_opmode_buffer)
    _, home_R1['orientation'] = sim.simxGetObjectOrientation(clientID, tip_R1, -1, sim.simx_opmode_buffer)
    home_R1['angles'] = inverse_kinematics(home_R1['position'], home_R1['orientation'], 'R1')
    _, home_R2['position'] = sim.simxGetObjectPosition(clientID, tip_R2, -1, sim.simx_opmode_buffer)
    _, home_R2['orientation'] = sim.simxGetObjectOrientation(clientID, tip_R2, -1, sim.simx_opmode_buffer)
    home_R2['angles'] = inverse_kinematics(home_R2['position'], home_R2['orientation'], 'R2')
    
    # Posición, orientación y ángulos de Checkpoints
    checkpoints_get_from_buffer([checkpoints_move_box, checkpoints_pioneer, checkpoints_boxes], 'R1', clientID)
    checkpoints_get_from_buffer([checkpoints_boxes_shelf, checkpoints_store_box, checkpoints_shelf], 'R2', clientID)
        
    print('Calculando trayectorias...')
    
    if USE_SAVED_TRAJECTORIES:
        with open('trajectories.p', 'rb') as fp:
            traj = pickle.load(fp)
    else:
        traj = calc_trajectories(home_R1, home_R2, checkpoints_move_box, checkpoints_pioneer, checkpoints_boxes, checkpoints_boxes_shelf, checkpoints_store_box, checkpoints_shelf)
        with open('trajectories.p', 'wb') as fp:
            pickle.dump(traj, fp, protocol=pickle.HIGHEST_PROTOCOL)


    sim.simxSetFloatSignal(clientID, 'beltVelocity', 0.5, sim.simx_opmode_oneshot)
    
    print('Iniciando simulación')
    
    boxes_type = []

    # Posicionar cajas en el Pioneer
    for i in range(BOX_COUNT):
        sensor_1_value = False
        while (sensor_1_value == False): # Se lee el sensor mientras no se detecte una caja
            _, sensor_1_value, _, _, _ = sim.simxReadProximitySensor(clientID, sensor_1, sim.simx_opmode_buffer)
            _, sensor_2_value, _, _, _ = sim.simxReadProximitySensor(clientID, sensor_2, sim.simx_opmode_streaming)
        
        sim.simxSetFloatSignal(clientID, 'beltVelocity', 0, sim.simx_opmode_oneshot)

        if sensor_2_value: # Caja grande
            boxes_type.append('large')
            move(traj['get_large_box'], joints_R1, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar
            move(traj['move_large_box'], joints_R1, clientID)
            move(traj['store_large_box_pioneer'][i], joints_R1, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar
            move(traj['back_home_large_box'][i], joints_R1, clientID)
        else: # Caja chica
            boxes_type.append('small')
            move(traj['get_small_box'], joints_R1, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar
            move(traj['move_small_box'], joints_R1, clientID)
            move(traj['store_small_box_pioneer'][i], joints_R1, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar
            move(traj['back_home_small_box'][i], joints_R1, clientID)

        sim.simxSetFloatSignal(clientID, 'beltVelocity', 0.5, sim.simx_opmode_oneshot)   

  

 



Conectado a  19999
Calculando trayectorias...
Iniciando simulación


In [4]:
# Mover robot
left_motor, right_motor = defineMotors(clientID)

proximity_l, proximity_f, proximity_f2, proximity_r = defineSensors(clientID)
vision_left, vision_center, vision_right = defineVisionSensors(clientID)

# Tiempo de espera para asegurarse que los valores leidos de home sean correctos
time.sleep(1)

firstSequence(clientID, vision_left, vision_center, vision_right, proximity_l, proximity_f, proximity_f2, proximity_r, left_motor, right_motor)
motorStop(clientID, left_motor=left_motor,
                right_motor=right_motor)
time.sleep(5*LOOP_DELAY)
# Mover robot

 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

moving backwards

 ---left: 0.054 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.056 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.057 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.058 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.061 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.062 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.064 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.065 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.067 ---front left: 0.000 ---front right: 0.000  ---right: 0.000 ---

turning left

 ---left: 0.070 ---front left: 0.000 ---front right: 0.000  -

TypeError: cannot unpack non-iterable bool object

In [ ]:
   
    # Posicionar cajas en los estantes  
    for i in range(BOX_COUNT):
        if boxes_type[i] == 'large':
            move(traj['pick_large_box'][i], joints_R2, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripperB_active', 1, sim.simx_opmode_oneshot) # Agarrar
            move(traj['move_to_shelf_large_box'][i], joints_R2, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripperB_active', 0, sim.simx_opmode_oneshot) # Soltar
            move(traj['back_home_from_shelf_large_box'][i], joints_R2, clientID)
        elif boxes_type[i] == 'small':
            move(traj['pick_small_box'][i], joints_R2, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripperB_active', 1, sim.simx_opmode_oneshot) # Agarrar
            move(traj['move_to_shelf_small_box'][i], joints_R2, clientID)
            sim.simxSetIntegerSignal(clientID, 'VacuumGripperB_active', 0, sim.simx_opmode_oneshot) # Soltar
            move(traj['back_home_from_shelf_small_box'][i], joints_R2, clientID)

    sim.simxFinish(clientID);
    print('Conexión Finalizada')

In [ ]:
with open('trajectories.p', 'rb') as fp:
    data = pickle.load(fp)


In [ ]:
sim.simxFinish(clientID);
print('Conexión Finalizada')